

#Book4 Ch10 10-7-1 to 10-7-2



#10-7-1 결정 트리, 랜덤 포레스트 모델

In [ ]:
!python --version
# 구글 코랩 파이썬 버전은 3.10.12입니다.
# 참고로 책을 저술하는 현재 시점에서 챗GPT가 사용하는 파이썬 버전은 3.8.10 입니다.

Python 3.10.12


In [ ]:
import sklearn

# 버전 확인
print(sklearn.__version__)
# 참고로 책을 저술하는 현재 시점에서 챗GPT가 사용하는 sklearn 버전은 1.0 입니다.

1.2.2


In [ ]:
import xgboost as xgb
print("XGBoost 버전:", xgb.__version__)
# 참고로 책을 저술하는 현재 시점에서 챗GPT가 사용하는 XGBoost 버전은 1.4.2 버전입니다.

XGBoost 버전: 2.0.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install xgboost

In [ ]:
pip install lightgbm

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Book4/Ch10/Bike_tree2.xlsx')

df.shape

(8755, 13)

In [ ]:
df.head(2)

,Month,Hour,Temperature,Humidity,Wind speed,Visibility,Solar Radiation,Seasons,Holiday,Functioning Day,Rented Bike B,Rainfall1,Snowfall1
0,12,0,0.220280,0.377551,0.360656,1.0,0.0,0,0,1,0,0,0
1,12,1,0.215035,0.387755,0.131148,1.0,0.0,0,0,1,0,0,0


In [ ]:
data = df.drop(['Rented Bike B'], axis=1)   # 타깃 변수를 제외한 설명 변수를 data에 저장
target = df['Rented Bike B']                # 타깃 변수만 target에 저장

# 50:50 데이터 변환
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.5, random_state=42)   # test_size=0.5임에 주의

#결정 트리 모델

In [ ]:
# 디폴트 모델
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
tree = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)

In [ ]:
# Decision Tree 모델 (Default인 GINI기준이자 Maximal depth 조건)
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=42)   # Classifier로 DecisionTreeClassifer 지정
tree.fit(X_train, y_train)   # Clssifier를 트레이닝 데이터셋에서 학습시킴

print("Accuracy(GINI) on training set:{:.5f}".format(tree.score(X_train, y_train)))
print("Accuracy(GINI) on test set:{:.5f}".format(tree.score(X_test, y_test)))

Accuracy(GINI) on training set:1.00000
Accuracy(GINI) on test set:0.90384


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeClassifier(random_state=42)

params = {'max_depth': [5, 10, 15, 20, 30, 40, 50]}

grid_tree = GridSearchCV(tree, param_grid=params, scoring='accuracy', cv=3, n_jobs=-1,
                     verbose=1)
grid_tree.fit(X_train, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_tree.best_score_))
print("GridSearchCV best parameter:", (grid_tree.best_params_))

Fitting 3 folds for each of 7 candidates, totalling 21 fits
GridSearchCV max accuracy:0.90724
GridSearchCV best parameter: {'max_depth': 10}


In [ ]:
# 최종 그리드 서치
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeClassifier(random_state=42)

params = {'max_depth': range(6, 15)}

grid_tree = GridSearchCV(tree, param_grid=params, scoring='accuracy', cv=3, n_jobs=-1,
                     verbose=1)
grid_tree.fit(X_train, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_tree.best_score_))
print("GridSearchCV best parameter:", (grid_tree.best_params_))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
GridSearchCV max accuracy:0.90861
GridSearchCV best parameter: {'max_depth': 7}


In [ ]:
model = grid_tree.best_estimator_
pred = model.predict(X_test)

print('r2: {0:.5f}'.format(accuracy_score(y_test, pred)))

r2: 0.91000


#랜덤 포레스트 모델

In [ ]:
# Random Forest 모델 (Default)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)   # Classifier로 DecisionTreeClassifer 지정
rf.fit(X_train, y_train)   # Clssifier를 트레이닝 데이터셋에서 학습시킴

print("Accuracy(GINI) on training set:{:.5f}".format(rf.score(X_train, y_train)))
print("Accuracy(GINI) on test set:{:.5f}".format(rf.score(X_test, y_test)))

Accuracy(GINI) on training set:1.00000
Accuracy(GINI) on test set:0.92462


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier(random_state=42)

params = {'max_depth':[5, 10, 15, 20, 30, 40, 50]}

grid_rf = GridSearchCV(rf, param_grid=params, scoring='accuracy', cv=3, n_jobs=-1,
                     verbose=1)
grid_rf.fit(X_train, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_rf.best_score_))
print("GridSearchCV best parameter:", (grid_rf.best_params_))

Fitting 3 folds for each of 7 candidates, totalling 21 fits
GridSearchCV max accuracy:0.92324
GridSearchCV best parameter: {'max_depth': 10}


In [ ]:
# 최종 그리드 서치
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier(random_state=42)

params = {'max_depth': range(6, 15)}

grid_rf = GridSearchCV(rf, param_grid=params, scoring='accuracy', cv=3, n_jobs=-1,
                     verbose=1)
grid_rf.fit(X_train, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_rf.best_score_))
print("GridSearchCV best parameter:", (grid_rf.best_params_))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
GridSearchCV max accuracy:0.92324
GridSearchCV best parameter: {'max_depth': 10}


In [ ]:
rf = RandomForestClassifier(random_state=42)
model = grid_rf.best_estimator_
pred = model.predict(X_test)

print('r2: {0:.5f}'.format(accuracy_score(y_test, pred)))

r2: 0.92417


#10-7-2 그레이디언트 부스티, XGBoost, LightGBM 모델

# 그레이디언트 부스팅 모델

In [ ]:
# Gradient Boosting 모델 (Default 모델)
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
gr = GradientBoostingClassifier(random_state = 42)
gr.fit(X_train, y_train)
pred = gr.predict(X_test)

print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

accuracy: 0.92394


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

# n_estimators는 GradientBoostingRegressor 디폴트 값 100으로 복귀
gr = GradientBoostingClassifier(random_state = 42)

parameters = {'learning_rate': [0.1],
              'max_depth': [5, 10, 15, 20],
              'n_estimators': [100],
              'subsample': [1]
              }

grid_gr = GridSearchCV(gr,
                        parameters,
                        scoring = 'accuracy',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_gr.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1], 'max_depth': [5, 10, 15, 20],
                         'n_estimators': [100], 'subsample': [1]},
             scoring='accuracy', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_gr.best_params_)

GridSearchCV 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1}


In [ ]:
model = grid_gr.best_estimator_
pred = model.predict(X_test)
print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

accuracy: 0.92714


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

# n_estimators는 GradientBoostingRegressor 디폴트 값 100으로 복귀
gr = GradientBoostingClassifier(random_state = 42)

parameters = {'learning_rate': [0.1],
              'max_depth': range(3,10),
              'n_estimators': [100],
              'subsample': [1]
              }

grid_gr = GridSearchCV(gr,
                        parameters,
                        scoring = 'accuracy',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_gr.fit(X_train, y_train)

Fitting 3 folds for each of 7 candidates, totalling 21 fits


GridSearchCV(cv=3, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1], 'max_depth': range(3, 10),
                         'n_estimators': [100], 'subsample': [1]},
             scoring='accuracy', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_gr.best_params_)

GridSearchCV 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1}


In [ ]:
model = grid_gr.best_estimator_
pred = model.predict(X_test)
print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

accuracy: 0.92714


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

# n_estimators는 GradientBoostingRegressor 디폴트 값 100으로 복귀
gr = GradientBoostingClassifier(random_state = 42)

parameters = {'learning_rate': [0.1],
              'max_depth': range(3,8),
              'n_estimators': [100],
              'subsample': [0.9, 1]
              }

grid_gr = GridSearchCV(gr,
                        parameters,
                        scoring = 'accuracy',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_gr.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


GridSearchCV(cv=3, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1], 'max_depth': range(3, 8),
                         'n_estimators': [100], 'subsample': [0.9, 1]},
             scoring='accuracy', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_gr.best_params_)

GridSearchCV 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.9}


In [ ]:
model = grid_gr.best_estimator_
pred = model.predict(X_test)
print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

accuracy: 0.92531


In [ ]:
# 최종 그리드 서치
from sklearn.model_selection import GridSearchCV

# n_estimators는 GradientBoostingRegressor 디폴트 값 100으로 복귀
gr = GradientBoostingClassifier(random_state = 42)

parameters = {'learning_rate': [0.01, 0.1],
              'max_depth': range(4,7),
              'n_estimators': [100],
              'subsample': [0.7, 0.8, 0.9]
              }

grid_gr = GridSearchCV(gr,
                        parameters,
                        scoring = 'accuracy',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_gr.fit(X_train, y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


GridSearchCV(cv=3, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1], 'max_depth': range(4, 7),
                         'n_estimators': [100], 'subsample': [0.7, 0.8, 0.9]},
             scoring='accuracy', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_gr.best_params_)

GridSearchCV 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.9}


In [ ]:
model = grid_gr.best_estimator_
pred = model.predict(X_test)
print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

accuracy: 0.92531


# XGBoost 모델

In [ ]:
# 기본 XGBClassifier 모델
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)

print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))


accuracy: 0.92599


In [ ]:
# 기본 XGBClassifier 모델
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# 책과 같이 챗GPT 용량에 맞춰서 n_estimators=10, max_depth=5 조건 추가
xgb = XGBClassifier(random_state=42, n_estimators=10, max_depth=5)
xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)

print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))
# 이 결과는 챗GPT에서의 결과와 다른데 이는 코랩과 챗GPT가 사용하는
# 파이썬 및 라이브러리 버전이 다른 경우 발생할 수 있습니다.
# 책에는 챗GPT의 결과 및 파이썬 결과를 병기해서 수록해 두었습니다.

accuracy: 0.92097


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

xgb = XGBClassifier(random_state=42)

parameters = {'colsample_bytree': [1],
              'learning_rate': [0.1],
              'max_depth': [5, 10, 15, 20],
              'min_child_weight' : [1],
              'n_estimators': [100],
              'subsample': [1]
              }

grid_xgb = GridSearchCV(xgb,
                        parameters,
                        scoring = 'accuracy',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_xgb.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [1], 'learning_rate': [0.1],
                         'max_depth': [5, 10, 15, 20], 'min_child_weight': [1],
                         'n_estimators': [100], 'subsample': [1]},
             scoring='accuracy', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_xgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1}


In [ ]:
model = grid_xgb.best_estimator_
pred = model.predict(X_test)

print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

accuracy: 0.92988


In [ ]:
# 최종 그리드 서치
from sklearn.model_selection import GridSearchCV

xgb = XGBClassifier(random_state=42)

parameters = {'colsample_bytree': [0.7, 0.8, 0.9, 1],
              'learning_rate': [0.01, 0.1],
              'max_depth': [5],
              'min_child_weight' : [1],
              'n_estimators': [100],
              'subsample': [0.7, 0.8, 0.7, 1]
              }

grid_xgb = GridSearchCV(xgb,
                        parameters,
                        scoring = 'accuracy',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_xgb.fit(X_train, y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.8, 0.9, 1],
                         'learning_rate': [0.01, 0.1], 'max_depth': [5],
                         'min_child_weight': [1], 'n_estimators': [100],
                         'subsample': [0.7, 0.8, 0.7, 1]},
             scoring='accuracy', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_xgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.7}


In [ ]:
model = grid_xgb.best_estimator_
pred = model.predict(X_test)

print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

accuracy: 0.93056


#LightGBM 모델

In [ ]:
# 기본 lGBMRegressor 모델
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

lgb = LGBMClassifier(random_state=42)
lgb.fit(X_train, y_train)
pred = lgb.predict(X_test)

print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2205, number of negative: 2172
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000786 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 945
[LightGBM] [Info] Number of data points in the train set: 4377, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503770 -> initscore=0.015079
[LightGBM] [Info] Start training from score 0.015079
accuracy: 0.92714


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)

# 워닝 메시지에서 2^max_depth > num_leaves를 권고해서 num_leaves 값을 1000로 정함
parameters = {'colsample_bytree': [1],
              'learning_rate': [0.1],
              'max_depth': [5, 10, 20, 30, 50, 50],
              'num_leaves': [1000],
              'min_child_weight' : [1],
              'n_estimators': [1000],
              'subsample': [1]
              }

grid_lgb = GridSearchCV(lgb,
                        parameters,
                        scoring = 'accuracy',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_lgb.fit(X_train, y_train)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_lgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 1000, 'num_leaves': 1000, 'subsample': 1}


In [ ]:
from sklearn.metrics import accuracy_score

model = grid_lgb.best_estimator_
pred = model.predict(X_test)

print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

accuracy: 0.92143


In [ ]:
# 최종 그리드 서치
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)

# 워닝 메시지에서 2^max_depth > num_leaves를 권고해서 num_leaves 값을 1000로 정함
parameters = {'colsample_bytree': [1],
              'learning_rate': [0.1],
              'max_depth': range(6, 21),
              'num_leaves': [1000],
              'min_child_weight' : [1],
              'n_estimators': [1000],
              'subsample': [1]
              }

grid_lgb = GridSearchCV(lgb,
                        parameters,
                        scoring = 'accuracy',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_lgb.fit(X_train, y_train)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_lgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 1000, 'num_leaves': 1000, 'subsample': 1}


In [ ]:
from sklearn.metrics import accuracy_score
model = grid_lgb.best_estimator_
pred = model.predict(X_test)
print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

accuracy: 0.92165


# 이하의 그리드 서치는 바로 위 최종 그리드 서치 결과를 능가하지 못함

In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)

# 워닝 메시지에서 2^max_depth > num_leaves를 권고해서 num_leaves 값을 1000로 정함
parameters = {'colsample_bytree': [0.7, 0.8, 0.9, 1],
              'learning_rate': [0.1],
              'max_depth': [6, 10, 20, 50],
              'num_leaves': [1000],
              'min_child_weight' : [1],
              'n_estimators': [1000],
              'subsample': [0.7, 0.8, 0.7, 1]
              }

grid_lgb = GridSearchCV(lgb,
                        parameters,
                        scoring = 'accuracy',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_lgb.fit(X_train, y_train)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_lgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 20, 'min_child_weight': 1, 'n_estimators': 1000, 'num_leaves': 1000, 'subsample': 0.7}


In [ ]:
from sklearn.metrics import accuracy_score
model = grid_lgb.best_estimator_
pred = model.predict(X_test)
print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

accuracy: 0.91914


In [ ]:
# 그리드 서치
# 런타임 2분 소요
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
# 워닝 메시지에서 2^max_depth > num_leaves를 권고해서 num_leaves 값을 1000로 정함
parameters = {'colsample_bytree': [0.7, 0.8, 0.9, 1],
              'learning_rate': [0.1],
              'max_depth': [20, 30, 40, 50, 100],
              'num_leaves': [1000],
              'min_child_weight' : [1],
              'n_estimators': [1000],
              'subsample': [0.7, 0.8, 0.9, 1]
              }

grid_lgb = GridSearchCV(lgb,
                        parameters,
                        scoring = 'accuracy',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_lgb.fit(X_train, y_train)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_lgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 20, 'min_child_weight': 1, 'n_estimators': 1000, 'num_leaves': 1000, 'subsample': 0.7}


In [ ]:
from sklearn.metrics import accuracy_score
model = grid_lgb.best_estimator_
pred = model.predict(X_test)
print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

accuracy: 0.91914


In [ ]:
# 그리드 서치
# 런타임 2분 소요
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
# 워닝 메시지에서 2^max_depth > num_leaves를 권고해서 num_leaves 값을 1000로 정함
parameters = {'colsample_bytree': [0.8, 0.9, 1],
              'learning_rate': [0.01, 0.1],
              'max_depth': [15, 20, 25],
              'num_leaves': [1000],
              'min_child_weight' : [1],
              'n_estimators': [1000],
              'subsample': [0.7, 0.8]
              }

grid_lgb = GridSearchCV(lgb,
                        parameters,
                        scoring = 'accuracy',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_lgb.fit(X_train, y_train)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_lgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 20, 'min_child_weight': 1, 'n_estimators': 1000, 'num_leaves': 1000, 'subsample': 0.7}


In [ ]:
from sklearn.metrics import accuracy_score
model = grid_lgb.best_estimator_
pred = model.predict(X_test)
print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

accuracy: 0.91914


In [ ]:
# 그리드 서치
# 런타임 2분 소요
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
# 워닝 메시지에서 2^max_depth > num_leaves를 권고해서 num_leaves 값을 1000로 정함
parameters = {'colsample_bytree': [0.9],
              'learning_rate': [0.1],
              'max_depth': range(16, 25),
              'num_leaves': [1000],
              'min_child_weight' : [1],
              'n_estimators': [1000],
              'subsample': [0.7]
              }

grid_lgb = GridSearchCV(lgb,
                        parameters,
                        scoring = 'accuracy',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_lgb.fit(X_train, y_train)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_lgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 20, 'min_child_weight': 1, 'n_estimators': 1000, 'num_leaves': 1000, 'subsample': 0.7}


In [ ]:
from sklearn.metrics import accuracy_score
model = grid_lgb.best_estimator_
pred = model.predict(X_test)
print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

accuracy: 0.91914


In [ ]:
# 그리드 서치
lgb = LGBMClassifier(random_state=42)
# 워닝 메시지에서 2^max_depth > num_leaves를 권고해서 num_leaves 값을 1000로 정함
parameters = {'colsample_bytree': [0.9],
              'learning_rate': [0.1],
              'max_depth': range(3, 16),
              'num_leaves': [1000],
              'min_child_weight' : [1],
              'n_estimators': [1000],
              'subsample': [0.7]
              }

grid_lgb = GridSearchCV(lgb,
                        parameters,
                        scoring = 'accuracy',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_lgb.fit(X_train, y_train)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_lgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 1000, 'num_leaves': 1000, 'subsample': 0.7}


In [ ]:
from sklearn.metrics import accuracy_score
model = grid_lgb.best_estimator_
pred = model.predict(X_test)
print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

accuracy: 0.92005


In [ ]:
# 그리드 서치
# 런타임 3분 소요
lgb = LGBMClassifier(random_state=42)
# 워닝 메시지에서 2^max_depth > num_leaves를 권고해서 num_leaves 값을 1000로 정함
parameters = {'colsample_bytree': [0.8, 0.9, 1],
              'learning_rate': [0.01, 0.1],
              'max_depth': range(3, 10),
              'num_leaves': [1000],
              'min_child_weight' : [1],
              'n_estimators': [1000],
              'subsample': [0.7, 0.8, 0.9]
              }

grid_lgb = GridSearchCV(lgb,
                        parameters,
                        scoring = 'accuracy',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_lgb.fit(X_train, y_train)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_lgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 0.9, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 1000, 'num_leaves': 1000, 'subsample': 0.7}


In [ ]:
from sklearn.metrics import accuracy_score
model = grid_lgb.best_estimator_
pred = model.predict(X_test)
print('accuracy: {0:.5f}'.format(accuracy_score(y_test, pred)))

accuracy: 0.92143
